In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.optimize as opt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/new-data/train.csv
/kaggle/input/new-data/test.csv
/kaggle/input/dont-overfit-ii/train.csv
/kaggle/input/dont-overfit-ii/sample_submission.csv
/kaggle/input/dont-overfit-ii/test.csv


In [53]:
train = pd.read_csv('/kaggle/input/new-data/train.csv')
train = train.set_index('id')
train.head()

,target,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [54]:
test = pd.read_csv('/kaggle/input/new-data/test.csv')
test = test.set_index('id')
test.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
250,0.500,-1.033,-1.595,0.309,-0.714,0.502,0.535,-0.129,-0.687,1.291,...,-0.088,-2.628,-0.845,2.078,-0.277,2.132,0.609,-0.104,0.312,0.979
251,0.776,0.914,-0.494,1.347,-0.867,0.480,0.578,-0.313,0.203,1.356,...,-0.683,-0.066,0.025,0.606,-0.353,-1.133,-3.138,0.281,-0.625,-0.761
252,1.750,0.509,-0.057,0.835,-0.476,1.428,-0.701,-2.009,-1.378,0.167,...,-0.094,0.351,-0.607,-0.737,-0.031,0.701,0.976,0.135,-1.327,2.463
253,-0.556,-1.855,-0.682,0.578,1.592,0.512,-1.419,0.722,0.511,0.567,...,-0.336,-0.787,0.255,-0.031,-0.836,0.916,2.411,1.053,-1.601,-1.529
254,0.754,-0.245,1.173,-1.623,0.009,0.370,0.781,-1.763,-1.432,-0.930,...,2.184,-1.090,0.216,1.186,-0.143,0.322,-0.068,-0.156,-1.153,0.825


In [55]:
test.insert(0, 'Ones', 1)
test.head()

,Ones,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
250,1,0.500,-1.033,-1.595,0.309,-0.714,0.502,0.535,-0.129,-0.687,...,-0.088,-2.628,-0.845,2.078,-0.277,2.132,0.609,-0.104,0.312,0.979
251,1,0.776,0.914,-0.494,1.347,-0.867,0.480,0.578,-0.313,0.203,...,-0.683,-0.066,0.025,0.606,-0.353,-1.133,-3.138,0.281,-0.625,-0.761
252,1,1.750,0.509,-0.057,0.835,-0.476,1.428,-0.701,-2.009,-1.378,...,-0.094,0.351,-0.607,-0.737,-0.031,0.701,0.976,0.135,-1.327,2.463
253,1,-0.556,-1.855,-0.682,0.578,1.592,0.512,-1.419,0.722,0.511,...,-0.336,-0.787,0.255,-0.031,-0.836,0.916,2.411,1.053,-1.601,-1.529
254,1,0.754,-0.245,1.173,-1.623,0.009,0.370,0.781,-1.763,-1.432,...,2.184,-1.090,0.216,1.186,-0.143,0.322,-0.068,-0.156,-1.153,0.825


In [56]:
train.insert(1, 'Ones', 1)
train.head()

,target,Ones,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,0.0,1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1.0,1,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1.0,1,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1.0,1,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [57]:
# set X and y (remember from above that we moved the label to column 0)
cols = train.shape[1]
X2 = train.iloc[:,1:cols]
y2 = train.iloc[:,0:1]
X2.head()

,Ones,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [58]:
X2.shape

(250, 301)

In [59]:
TX2 = test.iloc[:,0:cols]
TX2.head()

,Ones,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
250,1,0.500,-1.033,-1.595,0.309,-0.714,0.502,0.535,-0.129,-0.687,...,-0.088,-2.628,-0.845,2.078,-0.277,2.132,0.609,-0.104,0.312,0.979
251,1,0.776,0.914,-0.494,1.347,-0.867,0.480,0.578,-0.313,0.203,...,-0.683,-0.066,0.025,0.606,-0.353,-1.133,-3.138,0.281,-0.625,-0.761
252,1,1.750,0.509,-0.057,0.835,-0.476,1.428,-0.701,-2.009,-1.378,...,-0.094,0.351,-0.607,-0.737,-0.031,0.701,0.976,0.135,-1.327,2.463
253,1,-0.556,-1.855,-0.682,0.578,1.592,0.512,-1.419,0.722,0.511,...,-0.336,-0.787,0.255,-0.031,-0.836,0.916,2.411,1.053,-1.601,-1.529
254,1,0.754,-0.245,1.173,-1.623,0.009,0.370,0.781,-1.763,-1.432,...,2.184,-1.090,0.216,1.186,-0.143,0.322,-0.068,-0.156,-1.153,0.825


In [60]:
X2.head()

,Ones,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [61]:
y2.head()

,target
id,
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0


In [62]:
# convert to numpy arrays and initalize the parameter array theta
X2 = np.array(X2.values)
y2 = np.array(y2.values)
theta2 = np.zeros(301)

In [63]:
TX2 = np.array(TX2.values)
TX2

array([[ 1.   ,  0.5  , -1.033, ..., -0.104,  0.312,  0.979],
       [ 1.   ,  0.776,  0.914, ...,  0.281, -0.625, -0.761],
       [ 1.   ,  1.75 ,  0.509, ...,  0.135, -1.327,  2.463],
       ...,
       [ 1.   , -0.554, -0.936, ...,  1.546,  0.759,  0.519],
       [ 1.   , -0.746,  1.205, ...,  1.319, -0.181,  0.689],
       [ 1.   ,  0.736, -0.216, ..., -0.268,  0.133,  0.826]])

In [64]:
X2

array([[ 1.   , -0.098,  2.165, ..., -0.414,  1.038, -1.065],
       [ 1.   ,  1.081, -0.973, ..., -1.099, -0.936,  0.973],
       [ 1.   , -0.523, -0.089, ...,  0.004,  0.8  , -1.211],
       ...,
       [ 1.   ,  1.411, -1.465, ...,  0.76 ,  0.168, -0.719],
       [ 1.   ,  0.62 ,  1.04 , ..., -0.805,  2.029, -0.423],
       [ 1.   ,  0.489,  0.403, ..., -1.454, -0.625,  1.474]])

In [65]:
y2

array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],

In [66]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [67]:
def costReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [68]:
def gradientReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

In [69]:
learningRate = 100000

In [70]:
costReg(theta2, X2, y2, learningRate)

0.6931471805599454

In [71]:
gradientReg(theta2, X2, y2, learningRate)


array([-1.40000e-01, -5.53740e-02,  3.92120e-02, -1.60260e-02,
       -5.86400e-03,  5.66820e-02,  2.40640e-02,  2.30300e-02,
       -1.28860e-02,  2.75320e-02,  3.70200e-02,  7.78000e-03,
        1.84000e-04, -7.35200e-03, -5.27540e-02, -2.59680e-02,
        4.86580e-02,  6.84660e-02, -3.88920e-02, -2.78680e-02,
        3.06920e-02,  1.66840e-02, -3.28400e-02,  6.18400e-03,
        2.76000e-02, -6.47180e-02, -5.63760e-02,  3.68840e-02,
       -3.04440e-02,  3.35880e-02, -3.52660e-02, -5.75080e-02,
        1.11080e-02,  1.36100e-02, -1.62934e-01,  4.25000e-03,
        1.00580e-02, -2.73020e-02,  2.34360e-02,  2.85100e-02,
        7.43080e-02,  4.84000e-04,  1.14760e-02, -1.50920e-02,
        5.11220e-02,  3.80000e-04,  3.53060e-02, -3.14540e-02,
       -1.33780e-02,  9.90600e-03,  4.24000e-03, -2.86020e-02,
       -2.67400e-03,  2.74160e-02, -1.47760e-02, -2.36900e-02,
        3.12840e-02,  1.47300e-02,  1.64600e-02,  6.89200e-03,
       -3.21500e-02, -2.00220e-02,  3.29840e-02,  3.321

In [72]:
result2 = opt.fmin_tnc(func=costReg, x0=theta2, fprime=gradientReg, args=(X2, y2, learningRate))
result2

(array([ 1.09380956e-03,  2.53788753e-07, -1.72944528e-07, -3.60301756e-08,
         2.73347911e-08, -2.77299045e-07, -1.12331454e-07, -1.34049366e-07,
         9.28506664e-09, -1.08951750e-07, -2.04916263e-07, -3.43816399e-08,
        -7.64342347e-08,  2.22710911e-08,  2.58044918e-07,  1.37864098e-07,
        -2.10165751e-07, -3.45531505e-07,  2.62586809e-07,  1.11342050e-07,
        -1.19950647e-07, -5.13061751e-09,  1.10495806e-07, -4.18963520e-08,
        -1.13359723e-07,  4.03067197e-07,  1.69311340e-07, -2.36448159e-07,
         1.21159924e-07, -1.01212047e-07,  1.34459157e-07,  3.13201017e-07,
        -4.22007413e-08, -7.14306279e-08,  8.86119564e-07, -8.43648893e-08,
        -3.50561195e-08,  1.34569425e-07, -1.78705613e-08, -9.68449042e-08,
        -3.32133383e-07, -8.69395828e-09, -1.74742236e-08,  9.15607991e-08,
        -2.81492014e-07,  1.79200195e-08, -2.05038561e-07,  1.74372561e-07,
         1.02587133e-07, -1.06234288e-07, -3.51009748e-08,  1.17975601e-07,
        -3.1

In [73]:
theta_min = np.matrix(result2[0])

In [74]:
len(result2[0])

301

In [75]:
submission = pd.read_csv('/kaggle/input/dont-overfit-ii/sample_submission.csv')
submission['target'] = sigmoid(TX2 * theta_min.T)

submission.to_csv('submission.csv', index=False)

submission.head(15)

,id,target
0,250,0.500274
1,251,0.500274
2,252,0.500273
3,253,0.500275
4,254,0.500273
5,255,0.500273
6,256,0.500272
7,257,0.500273
8,258,0.500273
9,259,0.500273
